In [24]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast

In [2]:
df = pd.read_csv("output/features_labeled_v2.csv")

In [ ]:
df['abnormalpattern_noerror_ranked'] = df['abnormalpattern_noerror_ranked'].apply(lambda x: ast.literal_eval(x))
df['abnormalpattern_noerror_ranked'] = df['abnormalpattern_noerror_ranked'].apply(lambda x: ast.literal_eval(x))

In [5]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [4]:
log_template_miner = get_miner()

In [43]:
def get_pattern_prompt(col):
    """
    Generate a prompt for the LLM to predict the root cause type based on the abnormal pattern information.
    """
    patterns_details = []
    for pattern in col:
        event_details = []
        event_details.append(
            f"Actual Event Trace: ({from_id_to_template(int(pattern['events'].split('_')[0]), log_template_miner)}) -> "
            f"({from_id_to_template(int(pattern['events'].split('_')[1]), log_template_miner)})"
        )
        event_details.append(f"Abnormal Score: {pattern['score']}")
        event_details.append(f"Depth: {pattern['deepth']}")
        event_details.append(f"Pod: {pattern['pod']}")
        if "events_expected" in pattern:
            event_details.append(
                f"Expected Event Trace: {from_id_to_template(int(pattern['events_expected'].split('_')[0]), log_template_miner)} -> {from_id_to_template(int(pattern['events_expected'].split('_')[1]), log_template_miner)}"
            )
        patterns_details.append("\n".join(event_details))

    prompt = "\n\n".join(patterns_details)
    return prompt

In [ ]:
df['abnormalpattern_noerror_ranked_prompt'] = df['abnormalpattern_noerror_ranked'].apply(get_pattern_prompt)
df['abnormalpattern_noerror_ranked_prompt'] = df['abnormalpattern_noerror_ranked'].apply(get_pattern_prompt)

In [46]:
pd.set_option('display.max_colwidth', None)
df[['abnormalpattern_noerror_ranked_prompt']]

,abnormalpattern_noerror_ranked_prompt
0,Actual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ProductCatalogService/GetProduct start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/Convert start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (CpuUsageRate(%) Service: alarm)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/GetSupportedCurrencies start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CartService/GetCart start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.RecommendationService/ListRecommendations start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.AdService/GetAds start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ShippingService/GetQuote start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CartService/AddItem start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ProductCatalogService/ListProducts start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm
1,Actual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ProductCatalogService/GetProduct start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/Convert start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (CpuUsageRate(%) Service: alarm)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CurrencyService/GetSupportedCurrencies start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.CartService/GetCart start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.RecommendationService/ListRecommendations start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.AdService/GetAds start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActual Event Trace: (frontend hipstershop.Frontend/Recv. end Service: frontend) -> (frontend hipstershop.ShippingService/GetQuote start Service: frontend)\nAbnormal Score: 1.0\nDepth: 1\nPod: frontend-579b9bff58-t2dbm\n\nActua